In [31]:
import pandas as pd
import numpy as np
import nltk
import re
import cleaner
import hazm



In [2]:
data = pd.read_csv('./culture.csv')

In [46]:
def delete_stop_words(word):    
    if len(word)<3:
        return False
    else:
        n = re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)
    return n

In [47]:
def clean_text(text):
    normalizer = hazm.Normalizer()
    text = normalizer.affix_spacing(text)
    sentence = nltk.word_tokenize(unicode(text,'utf-8'))
    clean_words = [] 
    for i in range(len(sentence)):
        res = delete_stop_words(sentence[i])
        if res:
            clean_words.append({'Position':i,'Word':res})
    return clean_words

In [48]:
text = data['mText'][0]
words = clean_text(text)

In [49]:
lemmet = hazm.Lemmatizer()
for i in words:
    i['Word'] = lemmet.lemmatize(i['Word'])

In [51]:
s = nltk.stem.WordNetLemmatizer()

In [66]:
word_list = pd.Series([i['Word'] for i in words])

In [85]:
print(word_list.value_counts())

این           0.040000
طنز           0.026667
فارس          0.026667
روال          0.026667
بررسی         0.026667
بیشتر         0.026667
استان         0.026667
عادی          0.026667
خود           0.026667
شبکه          0.026667
شبخند         0.026667
شما           0.026667
گزارش         0.026667
ساعت          0.013333
فاصله         0.013333
شاعر          0.013333
آثار          0.013333
خبرگزاری      0.013333
پرداخت        0.013333
طنزپرداز      0.013333
برنامه        0.013333
پیام          0.013333
رفت#رو        0.013333
دوشنبه        0.013333
یاد           0.013333
پیش           0.013333
بنا           0.013333
خواست#خواه    0.013333
زاویه         0.013333
گذشت#گذر      0.013333
                ...   
درگذشت        0.013333
مدعو          0.013333
فاخر          0.013333
اخبار         0.013333
استاد         0.013333
مدت           0.013333
ادبیات        0.013333
رأس           0.013333
سازندگان      0.013333
ابوالفضل      0.013333
سرشناس        0.013333
نگاه          0.013333
فکاهی      

In [50]:
pd.Series(words)

0         {u'Position': 1, u'Word': u'گزارش'}
1      {u'Position': 2, u'Word': u'خبرگزاری'}
2          {u'Position': 3, u'Word': u'فارس'}
3           {u'Position': 5, u'Word': u'حال'}
4         {u'Position': 9, u'Word': u'بیشتر'}
5         {u'Position': 11, u'Word': u'آغاز'}
6       {u'Position': 12, u'Word': u'مسابقه'}
7          {u'Position': 13, u'Word': u'ادب'}
8        {u'Position': 14, u'Word': u'فکاهی'}
9        {u'Position': 15, u'Word': u'شبخند'}
10        {u'Position': 17, u'Word': u'شبکه'}
11         {u'Position': 18, u'Word': u'شما'}
12       {u'Position': 19, u'Word': u'بیشتر'}
13    {u'Position': 20, u'Word': u'گذشت#گذر'}
14    {u'Position': 21, u'Word': u'سازندگان'}
15         {u'Position': 22, u'Word': u'این'}
16      {u'Position': 23, u'Word': u'برنامه'}
17      {u'Position': 25, u'Word': u'احترام'}
18      {u'Position': 26, u'Word': u'درگذشت'}
19        {u'Position': 27, u'Word': u'زنده'}
20         {u'Position': 28, u'Word': u'یاد'}
21    {u'Position': 29, u'Word': u